# Telco Customer Churn — **02_Modeling_Interpretation**
*Prepared 2025-11-04 09:25 (WIB)*  
*Author: Hans Christian*

**Goal:** Build robust churn models with clear business interpretation.


## Block 0 — Setup & Paths

In [ ]:
!pip -q install imbalanced-learn xgboost shap

In [ ]:
# --- Imports dulu (sebelum pakai os) ---
import os, joblib
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

import matplotlib.pyplot as plt

# --- Colab mount & base path ---
try:
    from google.colab import drive  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    drive.mount('/content/drive')
    BASE = '/content/drive/MyDrive/Portofolio/1_Telco Customer Churn'
else:
    BASE = '/content/drive/MyDrive/Portofolio/1_Telco Customer Churn'  # sesuaikan jika lokal

CLEAN_FILE = f"{BASE}/data_clean/churn_clean.csv"
MODEL_DIR  = f"{BASE}/models"
os.makedirs(MODEL_DIR, exist_ok=True)

# --- Load data bersih ---
df = pd.read_csv(CLEAN_FILE)
print(df.shape)
df.head()

## Block 1 — Feature Split & Preprocess

In [ ]:

y = df["Churn"].map({"Yes":1, "No":0}).astype(int)
X = df.drop(columns=["Churn", "customerID"], errors="ignore")

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(include="object").columns.tolist()

pre = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Train:", X_train.shape, "Test:", X_test.shape)


## Block 2 — Baseline: Logistic Regression (with SMOTE)

In [ ]:

pipe = ImbPipeline(steps=[
    ("pre", pre),
    ("smote", SMOTE(random_state=42)),
    ("clf", LogisticRegression(max_iter=200, n_jobs=None))
])

pipe.fit(X_train, y_train)
preds = pipe.predict(X_test)
proba = pipe.predict_proba(X_test)[:,1]

print(classification_report(y_test, preds, digits=4))
print("ROC AUC:", roc_auc_score(y_test, proba))

RocCurveDisplay.from_predictions(y_test, proba)
plt.title("ROC Curve — Logistic Regression (SMOTE)")
plt.show()

joblib.dump(pipe, f"{MODEL_DIR}/logreg_smote.joblib")
print("Saved model:", f"{MODEL_DIR}/logreg_smote.joblib")


## Block 3 — Gradient Boosting (XGBoost)

In [ ]:

from xgboost import XGBClassifier

xgb = ImbPipeline(steps=[
    ("pre", pre),
    ("smote", SMOTE(random_state=42)),
    ("clf", XGBClassifier(
        n_estimators=400, max_depth=4, learning_rate=0.08, subsample=0.9, colsample_bytree=0.9,
        reg_lambda=1.0, random_state=42, eval_metric="logloss", n_jobs=-1
    ))
])

xgb.fit(X_train, y_train)
preds = xgb.predict(X_test)
proba = xgb.predict_proba(X_test)[:,1]

print(classification_report(y_test, preds, digits=4))
print("ROC AUC:", roc_auc_score(y_test, proba))
RocCurveDisplay.from_predictions(y_test, proba)
plt.title("ROC Curve — XGBoost (SMOTE)")
plt.show()

joblib.dump(xgb, f"{MODEL_DIR}/xgb_smote.joblib")
print("Saved model:", f"{MODEL_DIR}/xgb_smote.joblib")


## Block 4 — Explainability SHAP KernelExplainer

In [ ]:
# --- Block 4 — Explainability (SHAP KernelExplainer, stable) ---

# 0) Install (sekali saja di atas notebook kalau belum)
# !pip -q install shap

import shap, numpy as np, pandas as pd
import matplotlib.pyplot as plt

# 1) Fungsi prediksi proba dari PIPELINE pada RAW FEATURES
#    (KernelExplainer akan memanggil fungsi ini berulang)
def predict_proba_raw(Xraw: np.ndarray):
    Xdf = pd.DataFrame(Xraw, columns=X_train.columns)  # pastikan kolom sama urutan
    return xgb.predict_proba(Xdf)[:, 1]                # pipeline xgb -> proba kelas "churn=1"

# 2) Pilih background sample kecil agar cepat (representatif)
rng = np.random.default_rng(42)
bg_n   = min(200, len(X_train))
test_n = min(100, len(X_test))

bg_idx    = rng.choice(X_train.index, size=bg_n, replace=False)
test_idx  = rng.choice(X_test.index,  size=test_n, replace=False)

X_bg    = X_train.loc[bg_idx].copy()
X_testS = X_test.loc[test_idx].copy()

# 3) Buat KernelExplainer (gunakan link='logit' biar selaras dengan proba)
explainer = shap.KernelExplainer(predict_proba_raw, X_bg, link="logit")

# 4) Hitung SHAP values untuk subset test
#    nsamples: makin besar makin akurat namun lebih lambat (100–300 biasanya cukup)
shap_values = explainer.shap_values(X_testS, nsamples=200)

# 5) Summary plot (bar & beeswarm) — GLOBAL importance
shap.summary_plot(shap_values, X_testS, plot_type="bar", show=True)
shap.summary_plot(shap_values, X_testS, show=True)

# 6) Waterfall untuk SATU pelanggan (LOCAL explanation)
i = 0  # ganti index lokal yang ingin kamu jelaskan
shap.waterfall_plot(shap.Explanation(
    values=shap_values[i],
    base_values=explainer.expected_value,
    data=X_testS.iloc[i],
    feature_names=X_testS.columns
), max_display=20)
plt.show()

print("Done. Kernel SHAP berjalan (subset). Kamu bisa ubah bg_n/test_n/nsamples untuk akurasi vs kecepatan.")

## 🧭 Block 5 — Business Interpretation & What-If

- Tulis insight utama: siapa yang berisiko churn? (contract=month-to-month, tenure rendah, charges tinggi, dsb.)
- Rekomendasi: diskon/retention offer untuk segmen risiko tinggi; auto-flag ke CRM; A/B test untuk konfirmasi dampak.


In [ ]:
!pip -q install streamlit shap

In [ ]:
%cd "/content/drive/MyDrive/Portofolio/1_Telco Customer Churn/dashboard"
!streamlit run churn_dashboard_app.py --server.port 8501 --server.address 0.0.0.0

In [ ]:
# Jalankan Streamlit dengan auto-deteksi External URL
!nohup streamlit run churn_dashboard_app.py --server.port 8501 --server.address 0.0.0.0 & sleep 5 && grep -o 'https://[0-9a-zA-Z.-]*\.gradio\.live' nohup.out

In [ ]:
!grep -o 'https://[0-9a-zA-Z.-]*\.gradio\.live' nohup.out